In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Date & time handling
from datetime import timedelta

In [ ]:
customers = pd.read_csv('data/olist_customers_dataset.csv')
orders = pd.read_csv('data/olist_orders_dataset.csv')
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
sellers = pd.read_csv('data/olist_sellers_dataset.csv')
payments = pd.read_csv('data/olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
geolocation = pd.read_csv('data/olist_geolocation_dataset.csv')
category_translation = pd.read_csv('data/product_category_name_translation.csv')

### check data loaded correctly

In [5]:
datasets = {
    "customers": customers,
    "orders": orders,
    "order_items": order_items,
    "products": products,
    "sellers": sellers,
    "payments": payments,
    "order_reviews": order_reviews,
    "geolocation": geolocation,
    "category_translation": category_translation
}

for name, df in datasets.items():
    print(f"{name}: {df.shape}")


customers: (99441, 5)
orders: (99441, 8)
order_items: (112650, 7)
products: (32951, 9)
sellers: (3095, 4)
payments: (103886, 5)
order_reviews: (99224, 7)
geolocation: (1000163, 5)
category_translation: (71, 2)


In [6]:
def inspect_data(df, name="DataFrame", key_cols=None):
    """Comprehensive inspection of a dataframe including datatypes, nulls, duplicates, and keys."""
    
    print(f"\n{'='*60}")
    print(f"INSPECTING: {name}")
    print(f"{'='*60}\n")
    
    # Basic info
    print(f"Shape: {df.shape[0]:,} rows x {df.shape[1]} columns\n")
    
    # Column overview
    print("Column Overview:")
    print(f"{'Column':<30} {'Type':<15} {'Nulls':<15} {'Unique':<10}")
    print("-" * 70)
    for col in df.columns:
        dtype = str(df[col].dtype)
        nulls = f"{df[col].isnull().sum()} ({df[col].isnull().sum()/len(df)*100:.1f}%)"
        unique = df[col].nunique()
        print(f"{col:<30} {dtype:<15} {nulls:<15} {unique:<10}")
    
    # Columns with missing data
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if len(missing) > 0:
        print(f"\n⚠️  Columns with missing data:")
        for col, count in missing.items():
            print(f"  • {col}: {count:,} ({count/len(df)*100:.1f}%)")
    
    # Duplicate rows
    dup_count = df.duplicated().sum()
    if dup_count > 0:
        print(f"\n⚠️  Duplicate rows: {dup_count:,} ({dup_count/len(df)*100:.1f}%)")
    
    # Key column duplicates (optional)
    if key_cols:
        for key in key_cols:
            if key in df.columns:
                dup_keys = df[key].duplicated().sum()
                if dup_keys > 0:
                    print(f"\n⚠️  Duplicate values in key column '{key}': {dup_keys:,} "
                          f"({dup_keys/len(df)*100:.1f}%)")
    
    # Potential datetime columns
    datetime_cols = [col for col in df.columns if 'date' in col.lower() or 'timestamp' in col.lower()]
    if datetime_cols:
        print("\n📅 Potential datetime columns detected:")
        for col in datetime_cols:
            print(f"  • {col} ({df[col].dtype})")
    
    # Sample data
    print("\nFirst 3 rows:")
    print(df.head(3))
    
    print("\n" + "="*60 + "\n")


# -----------------------------
# Usage: Inspect all datasets consistently
# -----------------------------

primary_keys = {
    'customers': ['customer_id'],
    'orders': ['order_id'],
    'order_items': ['order_item_id', 'order_id'],
    'products': ['product_id'],
    'sellers': ['seller_id'],
    'payments': ['payment_sequential', 'order_id'],
    'order_reviews': ['review_id', 'order_id'],
    'geolocation': ['geolocation_zip_code_prefix'],
    'category_translation': ['product_category_name']
}

# Inspect all datasets
for name, df in datasets.items():
    keys = primary_keys.get(name, None)
    inspect_data(df, name, key_cols=keys)



INSPECTING: customers

Shape: 99,441 rows x 5 columns

Column Overview:
Column                         Type            Nulls           Unique    
----------------------------------------------------------------------
customer_id                    object          0 (0.0%)        99441     
customer_unique_id             object          0 (0.0%)        96096     
customer_zip_code_prefix       int64           0 (0.0%)        14994     
customer_city                  object          0 (0.0%)        4119      
customer_state                 object          0 (0.0%)        27        

First 3 rows:
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409         

### Data Inspection Summary

- **Orders:** Some missing values (`order_approved_at`, `order_delivered_carrier_date`, `order_delivered_customer_date`). All date/timestamp columns need conversion to `datetime`.  
- **Order Items:** Duplicate keys exist (`order_item_id` resets per order; multi-item orders cause `order_id` duplicates). `shipping_limit_date` requires datetime conversion.  
- **Products:** Small percentage of missing values in `product_category_name` and numeric columns (1.9%).  
- **Payments:** Duplicate keys are expected due to multiple payments per order. No missing values.  
- **Order Reviews:** Missing text fields (`review_comment_title`, `review_comment_message`). Minor duplicate keys exist. Datetime columns need conversion.  
- **Geolocation:** High row and key duplication due to multiple lat/lng entries per zip code.  
- **Customers, Sellers, Category Translation:** Clean, no missing values or duplicates.  


### Data Cleaning

In [7]:
# -----------------------------
# 1️⃣ Convert all datetime columns in datasets
# -----------------------------

# Define datetime columns per table
datetime_columns = {
    "orders": [
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date"
    ],
    "order_items": [
        "shipping_limit_date"
    ],
    "order_reviews": [
        "review_creation_date",
        "review_answer_timestamp"
    ]
}

# Convert columns to datetime
for table_name, cols in datetime_columns.items():
    df = datasets[table_name]  # get dataframe
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')  # coerce invalid formats to NaT
            print(f"{table_name} -> {col}: converted to datetime, nulls after conversion = {df[col].isnull().sum()}")
    datasets[table_name] = df  # update dataframe in datasets dict

# Quick check
for table_name in datetime_columns.keys():
    df = datasets[table_name]
    print(f"\n{table_name} datetime columns:")
    for col in datetime_columns[table_name]:
        print(f"  • {col}: dtype = {df[col].dtype}, nulls = {df[col].isnull().sum()}")


orders -> order_purchase_timestamp: converted to datetime, nulls after conversion = 0
orders -> order_approved_at: converted to datetime, nulls after conversion = 160
orders -> order_delivered_carrier_date: converted to datetime, nulls after conversion = 1783
orders -> order_delivered_customer_date: converted to datetime, nulls after conversion = 2965
orders -> order_estimated_delivery_date: converted to datetime, nulls after conversion = 0
order_items -> shipping_limit_date: converted to datetime, nulls after conversion = 0
order_reviews -> review_creation_date: converted to datetime, nulls after conversion = 0
order_reviews -> review_answer_timestamp: converted to datetime, nulls after conversion = 0

orders datetime columns:
  • order_purchase_timestamp: dtype = datetime64[ns], nulls = 0
  • order_approved_at: dtype = datetime64[ns], nulls = 160
  • order_delivered_carrier_date: dtype = datetime64[ns], nulls = 1783
  • order_delivered_customer_date: dtype = datetime64[ns], nulls = 2

In [8]:
# -----------------------------
# Check datatypes and sample data for all datasets
# -----------------------------

for name, df in datasets.items():
    print(f"\n{'='*60}")
    print(f"TABLE: {name} | Shape: {df.shape[0]:,} rows x {df.shape[1]} columns")
    print(f"{'='*60}\n")
    
    # Show datatypes
    print("Column datatypes:")
    print(df.dtypes)
    
    # Show first 5 rows
    print("\nFirst 5 rows:")
    print(df.head())
    
    print("\n" + "-"*60 + "\n")



TABLE: customers | Shape: 99,441 rows x 5 columns

Column datatypes:
customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

First 5 rows:
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409                 franca             SP  
1                      9790  sao bernardo do campo             SP  
2                      1151              sao paulo             SP  
3              

In [ ]:
# ==========================
# CLEANING ORDERS TABLE
# ==========================

# Check missing values before cleaning
print("Missing values in 'orders' before cleaning:")
print(orders[['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']].isnull().sum())

# Step 1: Handle missing 'order_approved_at'
# Usually these are very few; we can leave them as NaT or filter if needed later
# Step 2: Handle missing delivery dates
# Some missing delivery dates may indicate canceled or pending orders
# Let’s mark those rows as 'canceled' in a new column

orders['order_status_clean'] = orders['order_status']
orders.loc[orders['order_delivered_customer_date'].isnull(), 'order_status_clean'] = 'canceled'

# Step 3: Optional: Fill missing 'approved_at' with 'purchase_timestamp' if status is delivered
mask_missing_approved = orders['order_approved_at'].isnull() & (orders['order_status_clean'] != 'canceled')
orders.loc[mask_missing_approved, 'order_approved_at'] = orders.loc[mask_missing_approved, 'order_purchase_timestamp']

# Step 4: Verify cleaning
print("\nMissing values after cleaning:")
print(orders[['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']].isnull().sum())

# Step 5: Quick check of status distribution
print("\nOrder status distribution after cleaning:")
print(orders['order_status_clean'].value_counts())
